In [1]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline

transform = make_column_transformer(
    (OneHotEncoder(), ["industry"]),
    remainder="passthrough"
)

model = LinearRegression(fit_intercept=False)
qt = QuantileTransformer(output_distribution="normal")
poly = PolynomialFeatures(degree=2, include_bias=False)

# Get Data

In [ ]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

df = pd.read_sql(
"""
select ticker, date, mve, acc, agr, beta, bm, ep, gma, idiovol, lev, mom12m, mom1m, 
    operprof, roeq, roic, roaq, retvol, saleinv, currat
from today
where price > 5
""", 
conn
)

In [ ]:
df = df.dropna()
conn.close()
df = df.set_index("ticker")
df = df.sort_values(by="mve")
df = df.iloc[:-500]

## If you can not access the today table I uploaded a previously saved today table for you to test your code

In [108]:
df = pd.read_csv('./today_table.csv')

In [109]:
df = df.dropna()
df = df.set_index("ticker")
df = df.sort_values(by="mve")
df = df.iloc[:-500]

In [110]:
features = ["roeq", "bm", "mom12m"]

# Do same transformation as your training data

In [111]:
def qt_df(d):
    x = qt.fit_transform(d)
    return pd.DataFrame(x, columns=d.columns, index=d.index)

In [112]:
df[features] = qt.fit_transform(df[features])

In [113]:
inds = pd.read_csv("siccodes12.csv", index_col="industry")

def industry(sic):
  try:
    return inds[(inds.start<=sic)&(sic<=inds.end)].index[0]
  except:
    return "Other"

df["industry"] = df.siccd.map(industry)
features.append("industry")

# Load your trained pipeline

In [114]:
import joblib
pipe = joblib.load('./my_pipe.joblib')

# Make predictions

In [115]:
df["predict"] = pipe.predict(df[features])

# Connect to your Alpaca account

In [116]:
from alpaca.trading.client import TradingClient

KEY = 'PKDG9RGA3BLL985UTBY8' # Insert your key here
SECRET_KEY = 'Va7lW3jh5JWVcSfXaznDOeEU8uwvFHpmBw3lfCPE' # Insert your secret key here

trading_client = TradingClient(KEY, SECRET_KEY, paper=True)
assets = trading_client.get_all_assets()

# Get tradable and shortable stocks

In [117]:
tradable = {x.symbol: dict(x)['tradable'] if 'tradable' in dict(x).keys() else None for x in assets}
tradable = pd.DataFrame(data=tradable.values(), index=tradable.keys(), columns=['tradable'])

shortable = {x.symbol: dict(x)['shortable'] if 'shortable' in dict(x).keys() else None for x in assets}
shortable = pd.DataFrame(data=shortable.values(), index=shortable.keys(), columns=['shortable'])



In [118]:
df = df.join(tradable)
df = df.join(shortable)

In [119]:
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest

data_client = StockHistoricalDataClient(KEY, SECRET_KEY)
params = StockLatestQuoteRequest(
  symbol_or_symbols=df.index.to_list()
)
quotes = data_client.get_stock_latest_quote(params)
ask = {x: quotes[x].ask_price for x in quotes}
ask = pd.DataFrame(data = ask.values(), index = ask.keys(), columns=['ask'])
bid = {x: quotes[x].bid_price for x in quotes}
bid = pd.DataFrame(data=bid.values(), index = bid.keys(), columns=['bid'])
df = df.join(ask)
df = df.join(bid)

In [120]:
account = trading_client.get_account()
equity = float(account.equity)

current = trading_client.get_all_positions()
if len(current) > 0:
  df["current"] = {x.symbol: int(x.qty) for x in current}
  df["current"] = df.current.fillna(0)
else:
  df["current"] = 0

In [126]:
df.dropna(inplace=True)

In [127]:
num_long = num_short = 100

best = df[df.tradable].sort_values(by="predict").iloc[-num_long:]
best["target"] = (equity / num_long) / best.ask

worst = df[df.shortable].sort_values(by="predict").iloc[:num_short]
worst["target"] = - (equity / num_short) / worst.bid

# Make trades
## You can only make trades when the market is open

In [128]:
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.trading.requests import MarketOrderRequest, GetAssetsRequest
from alpaca.trading.enums import OrderSide, TimeInForce, AssetClass

In [ ]:
for tick in worst.index: 
    if worst.loc[tick, "target"]<0:
        try:
            market_order_data = MarketOrderRequest(
                symbol=tick,
                qty=-worst.loc[tick, "target"],
                side=OrderSide.SELL,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )
            print(market_order)
        except Exception as e:
            print(e)
            print(f"sell order for {tick} failed")

for tick in best.index: 
    if best.loc[tick, "target"]>0:
        try:
            market_order_data = MarketOrderRequest(
                symbol=tick,
                qty=best.loc[tick, "target"],
                side=OrderSide.BUY,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )
            print(market_order)
        except Exception as e:
            print(e)
            print(f"buy order for {tick} failed")